In [ ]:
# !pip install braindecode
# !pip install mne

In [ ]:
import braindecode

In [ ]:
import mne
from scipy.io import loadmat
import scipy
import sklearn
import numpy as np
import pandas as pd
import glob
from mne.decoding import CSP
import os

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, LeaveOneGroupOut, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda

In [ ]:
import warnings
warnings.filterwarnings('ignore') # to ignore warnings

In [ ]:
verbose = False                    # global variable to suppress output display of MNE functions
mne.set_log_level(verbose=verbose) # to suppress large info outputs

In [ ]:
n_jobs = -1  # for multicore parallel processing, set it to 1 if cause memory issues, for full utilization set to -1

## Data Loading and Conversion to MNE Datatypes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = "/content/drive/MyDrive/BCI/AI/Stroke_Notebooks/Data"

In [ ]:
training_files   = glob.glob(folder_path + '/*T.mat')
len(training_files)     # if  return zero,then no file is loaded

8

## Lets Append Epochs

In [ ]:
def get_mne_epochs_complete(files_paths, verbose=verbose, t_start=2, fs=512, mode='train'):
    '''
    similar to get_mne_epochs, just appends data from all relevant files together to give a single
    epoch object
    '''
    eeg_data = []
    for filepath in files_paths:
        mat_data = loadmat(filepath)
        eeg_data.extend(mat_data['RawEEGData'])

    idx_start = fs*t_start      # fs*ts
    eeg_data = np.array(eeg_data)
    eeg_data = eeg_data[:, :, idx_start:]
    event_id = {'left-hand': 0, 'right-hand': 1} # pytorch expects labels in [0, n_classes-1]
    channel_names = ['F3', 'FC3', 'C3', 'CP3', 'P3', 'FCz', 'CPz', 'F4', 'FC4', 'C4', 'CP4', 'P4']
    info = mne.create_info(ch_names=channel_names, sfreq=fs, ch_types='eeg')
    epochs = mne.EpochsArray(eeg_data, info, verbose=verbose, tmin=t_start-3.0)
    epochs.set_montage('standard_1020')
    epochs.filter(1., None) # required be ICA, (7-30 Hz) later
    epochs.apply_baseline(baseline=(-.250, 0)) # linear baseline correction

    if mode == 'train': # this in only applicable for training data
        labels = []
        for filepath in files_paths:
            mat_data = loadmat(filepath)
            labels.extend(mat_data['Labels'].ravel() - 1)
        epochs.event_id = event_id
        epochs.events[:,2] = labels
    return epochs

### Data Loading with Band Pass Filtering

In [ ]:
# loading relevant files
training_epochs_all = get_mne_epochs_complete(training_files).filter(7,32) # for all training subjects

In [ ]:
epochs = training_epochs_all.copy()
data, labels = epochs.get_data(), epochs.events[:,-1]
print('Shape of EEG Data: ', data.shape, '\t Shape of Labels: ', labels.shape)

Shape of EEG Data:  (640, 12, 3072) 	 Shape of Labels:  (640,)


In [ ]:
data[0].shape

(12, 3072)

In [ ]:
data[0,0]

array([-15.78670162, -17.63311493, -18.00123611, ..., -13.69729648,
        -2.63265778,   8.42215194])

## Deep Learning with Braindecode

### It's Training Time with [0.5, 4.5] sec and 2sec window with 1 sec stride (using leave one group out cv)

In [ ]:
epochs = training_epochs_all.copy()
epochs = epochs.crop(tmin=0.5, tmax=4.5, include_tmax=False)

In [ ]:
from braindecode.datautil import create_from_mne_epochs

# convert epochs to braindecode compatible datastructure
# 2sec windows with 0.125 sec stride
window_size = 1024 #1024 #1024 #50 # 3072
window_stride = 512 #512 #256 # 50

windows_datasets = create_from_mne_epochs(
            [epochs], # expects list of epochs
            window_size_samples = window_size,
            window_stride_samples = window_stride,
            drop_last_window = False
)

In [ ]:
def get_windows_datasets_labels(windows_dataset):
    labels = []
    for i in range(len(windows_dataset.datasets)):
        labels.extend(windows_dataset.datasets[i].y)
    return np.array(labels)

In [ ]:
windows_description = pd.DataFrame(data=get_windows_datasets_labels(windows_datasets),
                                           columns=['labels'])

In [ ]:
windows_description

,labels
0,0
1,0
2,0
3,0
4,0
...,...
1915,0
1916,0
1917,0
1918,0


In [ ]:
windows_datasets.set_description = windows_description

In [ ]:
windows_datasets.set_description.labels[500:]

500     1
501     1
502     1
503     1
504     1
       ..
1915    0
1916    0
1917    0
1918    0
1919    0
Name: labels, Length: 1420, dtype: int64

In [ ]:
print("Total Windows in a whole Dataset: ", len(windows_datasets.set_description))

Total Windows in a whole Dataset:  1920


In [ ]:
from braindecode.datautil.preprocess import exponential_moving_standardize

In [ ]:
from braindecode.datautil.preprocess import preprocess
# from braindecode.datautil.preprocess import MNEPreproc, NumpyPreproc, preprocess

In [ ]:
from braindecode.preprocessing import (
    exponential_moving_standardize,
    preprocess,
    Preprocessor,
)

In [ ]:
low_cut_hz = 8.  # low cut frequency for filtering
high_cut_hz = 32.  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

In [ ]:
def custom_exp_moving_std_fn(epochs, factor_new=factor_new, init_block_size=init_block_size):
    data = epochs.get_data()
    for i in range(len(data)):
        epochs._data[i] = exponential_moving_standardize(data[i],
                        factor_new=factor_new, init_block_size=init_block_size)
    return epochs

In [ ]:
preprocessors = [
    Preprocessor("pick_types", eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(
        lambda data, factor: np.multiply(data, factor),  # Convert from V to uV
        factor=1e6,
    ),
    Preprocessor("filter", l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    # Preprocessor(
    #     # fn=custom_exp_moving_std_fn,  # Exponential moving standardization
    #     factor_new=factor_new,
    #     init_block_size=init_block_size,
    # ),
]

In [ ]:
preprocess(windows_datasets, preprocessors)
# preprocess(windows_datasets, preprocessors, n_jobs=-1)

In [ ]:
import tensorflow as tf

# Check if GPU is available
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

# Print GPU specifications
!nvidia-smi


Default GPU Device: /device:GPU:0
Wed Feb  7 19:44:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              25W /  70W |    103MiB / 15360MiB |      3%      Default |
|                                         |                      |                  N/A |
+---------------------------------

In [ ]:
# Creating a model
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet, EEGNetv4


In [ ]:
from braindecode.models import EEGInceptionMI

In [ ]:
cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 20200220  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed, cuda=cuda)

In [ ]:
n_classes=2
# Extract number of chans and time steps from dataset
n_chans = windows_datasets[0][0].shape[0]
input_window_samples = windows_datasets[0][0].shape[1]

In [ ]:
n_chans

12

In [ ]:
input_window_samples

1024

In [ ]:
# model = EEGNetv4(
#     n_chans,
#     n_classes,
#     input_window_samples = window_size, #input_window_samples,
#     final_conv_length='auto',
# )
model = EEGInceptionMI(
    n_chans,
    n_classes,
    input_window_seconds = 2,
    sfreq  = 512
)

# Send model to GPU
if cuda:
    model.cuda()

In [ ]:
# cv = LeaveOneGroupOut()
# # group parameter for leave one group out cross validation in sklearn, each subject is given unique identifier
# group_list = []
# for subject in np.linspace(1,8,8):
#     group_list.extend([subject for _ in range(len(windows_datasets)//8)]) #since total 8 subjects
# groups = np.array(group_list)

In [ ]:
# Training time
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier

In [ ]:
import skorch

In [ ]:
# lr = 1 * 0.02 #0.01
# weight_decay = 0.5 * 0.001
# batch_size = 32 #64
# n_epochs = 25 #25 #25 #25 #25 #20 #25 use few epochs for quick verification

In [ ]:
lr =  0.01
weight_decay = 0
batch_size = 64
n_epochs = 50#25 #25 #25 #25 #25 #20 #25 use few epochs for quick verification

In [ ]:
clf =  EEGClassifier(
                    model,
                    criterion=torch.nn.NLLLoss,
                    optimizer=torch.optim.AdamW,
                    #iterator_train = StratifiedShuffleSplit(),#cv.split(epochs, y=labels, groups=groups),
                    #train_split=predefined_split(train_set),  # using valid_set for validation
                    #train_split = cv.split(epochs, y=labels, groups=groups),
                    train_split = None,
                    optimizer__lr=lr,
                    optimizer__weight_decay=weight_decay,
                    batch_size=batch_size,
                    callbacks=[
                        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
                    ],
                    device=device,
                    # classes=classes,

                    )

In [ ]:
dataset = windows_datasets
clf.fit(dataset, y=dataset.set_description.labels, epochs=n_epochs)

  epoch    train_accuracy    train_loss      lr       dur
-------  ----------------  ------------  ------  --------
      1            0.5750        0.7506  0.0100  123.2320
      2            0.6771        0.5692  0.0100  105.3215
      3            0.7422        0.6013  0.0100  105.9259
      4            0.6031        0.5616  0.0099  105.9321
      5            0.7609        0.4820  0.0098  105.7254
      6            0.7427        0.4572  0.0097  105.9933
      7            0.7557        0.4901  0.0096  105.9943
      8            0.6036        0.5055  0.0095  105.9141
      9            0.8161        0.3339  0.0094  105.8824
     10            0.8479        0.2266  0.0092  105.8364
     11            0.5880        0.1870  0.0090  105.9988
     12            0.9474        0.1534  0.0088  105.9205
     13            0.9677        0.1395  0.0086  105.8701
     14            0.9276        0.0595  0.0084  105.9529
     15            0.9115        0.0392  0.0081  106.0809


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=================================================================================================================================================================
  Layer (type (var_name):depth-idx)                            Input Shape               Output Shape              Param #                   Kernel Shape
  EEGInceptionMI (EEGInceptionMI)                              [1, 12, 1024]             [1, 2]                    --                        --
  ├─Ensure4d (ensuredims): 1-1                                 [1, 12, 1024]             [1, 12, 1024, 1]          --                        --
  ├─Rearrange (dimshuffle): 1-2                                [1, 12, 1024, 1]          [1, 12, 1, 1024]          --                        --
  ├─_ResidualModuleMI (residual_block_1): 1-3                  [1, 12, 1, 1024]          [1, 288, 1, 1024]         --                        --
  │    └─Conv2d (conv): 2-1           

In [ ]:
dataset

In [ ]:
def training_function(windows_datasets, n_epochs=25):
    print('\n', '#'*25, 'Cross Subject Training:', '#'*25, '\n')
    dataset = windows_datasets
    clf.fit(dataset, y=dataset.set_description.labels, epochs=n_epochs);
    best_validation_acc = clf.callbacks_[4][1].best_score_ # a hack to get best validation accuracy
    best_validation_kappa = (2*best_validation_acc)-1
    print("Best Cross Validation Kappa Score: {:.2f}".format(best_validation_kappa))

In [ ]:
training_function(windows_datasets, n_epochs=n_epochs);


 ######################### Cross Subject Training: ######################### 

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.


RuntimeError: Given input size: (288x1x1024). Calculated output size: (288x1x0). Output size is too small